# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [32]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [33]:
input_data='./input_data/'
#output_data='./output_data/2020/'
output_data='./output_data/'

In [34]:
model_name='Base Case'#'1 Unit Available' MW153#MW144 'MW162'#'Fixed Tailwater'#'Base Case' Headpond50cm EWRAllCs
dryrun=False #Dont write output files

In [35]:


models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
#models=pd.read_csv(input_data + "2020/models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="/scenarios/" + out_prefix

category_set_name=model['EWRCatSet']#'Recommendation 1'
headpond_lift=model['HeadpondLift']#0
plant_capacity=model['Capacity'] #Base 180MW nominal capacity. Only used to calculate 'nominal capacity factor'
units=4 #Design no of units
units_avialable=model['UnitsAvailable'] #Available no of units - to see what happens during testing/commissioning

rated_flow_unit=model['RatedDischarge'] #250
max_flow_unit=rated_flow_unit*model['MaximumLoadFactor'] #275
canal_capacity=max_flow_unit*4 #canal capacity 1100 cumec 
min_flow_unit=model['MinimumFlowUnit'] #50cumec minimum flow for a turbine
fixed_tailwater_level=model['FixedTailwater']#False #970 
head_minimum=model['MinimumHead'] #minimum head 10m

rated_power_unit=rated_flow_unit*model['RatedHead']*model['RatedTurbineEfficiency']*model['RatedGeneratorEfficiency']*9.81/1000

model

OutputPrefix                            base
Description                        Base Case
EWRCatSet                   Recommendation 1
HeadpondLift                               0
Capacity                                 180
MinimumHead                               10
MinimumFlowUnit                           50
FixedTailwater                             0
UnitsAvailable                             4
RatedDischarge                           250
RatedHead                               21.7
RatedTurbineEfficiency                 0.943
RatedGeneratorEfficiency                0.98
MaximumLoadFactor                        1.1
RatedPower                            49.182
Mean                                  832484
P95                                   718504
P90                                   756610
P75                                   792216
P50                                   838165
P25                                   876248
P10                                   921915
P5        

## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [36]:
daily=pd.read_csv(output_data + out_prefix + '_pe_daily.csv') #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)

In [37]:
daily=daily.drop(daily.loc['2020-05-01':].index)
daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33968 entries, 1924-10-01 to 2017-09-30
Data columns (total 84 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   LaggedDate                  33968 non-null  object 
 1   VicFalls                    33968 non-null  float64
 2   Conversion                  33968 non-null  float64
 3   Flow                        33968 non-null  float64
 4   Exceedance                  33968 non-null  float64
 5   Year                        33968 non-null  int64  
 6   Month                       33968 non-null  int64  
 7   Day                         33968 non-null  int64  
 8   MonthId                     33968 non-null  float64
 9   WaterYear                   33968 non-null  int64  
 10  WaterMonth                  33968 non-null  int64  
 11  WaterDay                    33968 non-null  int64  
 12  WaterWeek                   33968 non-null  int64  
 13  Volume        

## Monthly


In [38]:

monthly=daily.groupby("MonthId").mean().drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

if 'VicFalls' in monthly.columns:
    monthly=monthly.drop(['VicFalls','Conversion','Volume','WaterDay','WaterWeek','Volume'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
   FlowAvailableForGeneration_min=('FlowAvailableForGeneration','min')
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1116 entries, 1924.1 to 2017.09
Data columns (total 80 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Flow                            1116 non-null   float64
 1   Exceedance                      1116 non-null   float64
 2   Year                            1116 non-null   float64
 3   Month                           1116 non-null   float64
 4   WaterYear                       1116 non-null   float64
 5   WaterMonth                      1116 non-null   float64
 6   Flow_difference                 1116 non-null   float64
 7   Flow_difference_pct             1116 non-null   float64
 8   EWRChannelA                     1116 non-null   float64
 9   EWRChannelC                     1116 non-null   float64
 10  EWRChannelD                     1116 non-null   float64
 11  EWRChannelE                     1116 non-null   float64
 12  EWRChannelFG            

## Yearly

In [39]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),   
   PowerTurbine1_max=('PowerTurbine1', 'max'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [40]:
yearly=yearly.drop(yearly.loc['2019':].index)
yearly.tail(5)

,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,PowerTurbine1_max,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor
WaterYear,,,,,,,,,,,,,
2012,3614.802471,223.273423,0.508220,0.055826,25.083495,13.168827,60.355148,974.894271,964.716509,191.310490,24.528540,843130.264390,0.534710
2013,3574.946119,195.061590,0.497279,0.056428,25.436464,13.247224,59.555521,974.815874,964.405522,191.927167,20.671377,926459.111352,0.587557
2014,1460.253290,208.780681,0.546606,0.099878,25.098362,18.623511,60.378249,969.484512,964.706890,192.407900,25.316443,884558.950509,0.560984
2015,2975.524395,176.218173,0.496598,0.067347,25.390405,14.500960,60.077094,973.562138,964.405522,191.268651,21.964965,786286.526245,0.498660
2016,3889.991232,156.550907,0.487279,0.047442,25.390405,12.642403,59.120258,975.420695,964.405522,192.177553,18.622058,851845.332264,0.540237


In [41]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))

In [42]:
yearly['PropDays1uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']>rated_power_unit).groupby('WaterYear').count()
yearly['PropDays2uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']>rated_power_unit*2).groupby('WaterYear').count()
yearly['PropDays3uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']>rated_power_unit*3).groupby('WaterYear').count()
yearly['PropDays4uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']>yearly['Power_max'].max()).groupby('WaterYear').count()

In [43]:
daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

WaterYear
1924.0     57
1925.0     66
1926.0     53
1927.0     67
1928.0    104
         ... 
2012.0     24
2013.0     47
2014.0    128
2015.0     61
2016.0     54
Name: Date, Length: 85, dtype: int64

In [44]:
yearly['FstDay1uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay1uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay2uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']<rated_power_unit*2).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay2uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']<rated_power_unit*2).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay3uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']<rated_power_unit*3).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay3uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']<rated_power_unit*3).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay4uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']<yearly['Power_max'].max()).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay4uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']<yearly['Power_max'].max()).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)


## Power and energy exceedance

In [45]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed.head(3)

,Power
Exceedance,
0.00,193.908405
0.05,181.679247
0.10,173.476231


In [46]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc.head(3)

,Energy
0.00,1.018971e+06
0.05,9.582529e+05
0.10,9.219147e+05


In [47]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [48]:
min_year=daily['Year'].min()
max_year=daily['Year'].max()


In [49]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([min_year,max_year],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(plant_capacity*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,854344.223735,0.541822
1926,779350.985117,0.494261
1927,854880.480412,0.542162
1928,829351.404158,0.525971
1929,753745.429409,0.478022


## Monthly Day
15th of every month

In [50]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [51]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-19,2017,9,19,2016,216,0.80,4,34,6,4,...,0.91,0.0,0.0,0.0,27.3,0.0,0.0,0.0,27.3,654
2017-09-20,2017,9,20,2016,213,0.80,4,34,6,4,...,0.91,0.0,0.0,0.0,26.6,0.0,0.0,0.0,26.6,638
2017-09-21,2017,9,21,2016,213,0.82,4,34,6,4,...,0.91,0.0,0.0,0.0,26.6,0.0,0.0,0.0,26.6,638
2017-09-22,2017,9,22,2016,209,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,25.5,0.0,0.0,0.0,25.5,612
2017-09-23,2017,9,23,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,23.8,0.0,0.0,0.0,23.8,572
2017-09-24,2017,9,24,2016,200,0.82,4,34,6,4,...,0.89,0.0,0.0,0.0,23.1,0.0,0.0,0.0,23.1,553
2017-09-25,2017,9,25,2016,200,0.82,4,34,6,4,...,0.89,0.0,0.0,0.0,23.1,0.0,0.0,0.0,23.1,553
2017-09-26,2017,9,26,2016,200,0.82,4,34,6,4,...,0.89,0.0,0.0,0.0,23.1,0.0,0.0,0.0,23.1,553
2017-09-27,2017,9,27,2016,200,0.82,4,34,6,4,...,0.89,0.0,0.0,0.0,23.1,0.0,0.0,0.0,23.1,553


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [52]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','WaterMonth','Year','MonthId','Day','WaterDay','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

#if 'VicFalls' in weekly.columns:
weekly=weekly.drop(['VicFalls','Conversion','Volume'],axis=1)

weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(plant_capacity*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  Flow_difference  \
WaterYear WaterWeek                                            
2016      42         451.943382    0.593143         6.536288   
          43         411.172524    0.638714         4.724796   
          44         377.834520    0.678857         5.261728   
          45         344.446201    0.716714         4.257422   
          46         322.314771    0.751857         2.882230   
          47         296.218764    0.791143         3.371720   
          48         271.256811    0.834000         3.832214   
          49         250.464258    0.873286         3.094980   
          50         226.821410    0.915571         2.855981   
          51         215.657828    0.933429         1.480060   
          52         200.905599    0.953571         1.303758   
          53         200.013130    0.958000         0.539477   

                     Flow_difference_pct  EWRChannelA  EWRChannelC  \
WaterYear WaterWeek                                                  
2016      42                    0.014538    50.000000    10.000000   
          43                    0.011456    50.000000    10.000000   
          44                    0.013976    43.714286     8.285714   
          45                    0.012366    39.000000     7.000000   
          46                    0.009020    39.000000     7.000000   
          47                    0.011315    39.000000     7.000000   
          48                    0.014055    38.285714     6.857143   
          49                    0.012470    34.000000     6.000000   
          50                    0.012523    34.000000     6.000000   
          51                    0.006946    34.000000     6.000000   
          52                    0.006443    34.000000     6.000000   
          53                    0.002697    34.000000     6.000000   

                     EWRChannelD  EWRChannelE  EWRChannelFG    EWRTotal  ...  \
WaterYear WaterWeek                                                      ...   
2016      42            9.000000     9.000000    108.000000  186.000000  ...   
          43            9.000000     9.000000    108.000000  186.000000  ...   
          44            6.714286     6.142857     80.571429  145.428571  ...   
          45            5.000000     4.000000     60.000000  115.000000  ...   
          46            5.000000     4.000000     60.000000  115.000000  ...   
          47            5.000000     4.000000     60.000000  115.000000  ...   
          48            4.857143     3.857143     57.714286  111.571429  ...   
          49            4.000000     3.000000     44.000000   91.000000  ...   
          50            4.000000     3.000000     44.000000   91.000000  ...   
          51            4.000000     3.000000     44.000000   91.000000  ...   
          52            4.000000     3.000000     44.000000   91.000000  ...   
          53            4.000000     3.000000     44.000000   91.000000  ...   

                     HeadTurbine1_max  HeadTurbine1_min  LevelTailwater_max  \
WaterYear WaterWeek                                                           
2016      42                24.149540         23.875651          965.589031   
          43                24.268319         24.044971          965.380858   
          44                24.397504         24.164017          965.236958   
          45                24.520135         24.355542          965.083218   
          46                24.660401         24.535417          964.993439   
          47                24.807667         24.704049          964.908128   
          48                24.940660         24.836336          964.840915   
          49                24.992931         24.902176          964.778028   
          50                25.067056         25.014660          964.730913   
          51                25.096172         25.067056          964.713474   
          52                25.115752         25.111050          964.706442   
 

In [53]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [54]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Power_P95']=monthly[['WaterMonth','Power']].groupby('WaterMonth').quantile(0.05)
calmonthly['Power_P90']=monthly[['WaterMonth','Power']].groupby('WaterMonth').quantile(0.10)

calmonthly['FlowAvailableForGeneration_P95']=monthly[['WaterMonth','FlowAvailableForGeneration']].groupby('WaterMonth').quantile(0.05)
calmonthly['FlowAvailableForGeneration_P50']=monthly[['WaterMonth','FlowAvailableForGeneration']].groupby('WaterMonth').quantile(0.5)

calmonthly['PotentialPower1u_P95']=monthly[['WaterMonth','PotentialPower1u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower1u_P50']=monthly[['WaterMonth','PotentialPower1u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower2u_P95']=monthly[['WaterMonth','PotentialPower2u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower2u_P50']=monthly[['WaterMonth','PotentialPower2u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower3u_P95']=monthly[['WaterMonth','PotentialPower3u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower3u_P50']=monthly[['WaterMonth','PotentialPower3u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower4u_P95']=monthly[['WaterMonth','PotentialPower4u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower4u_P50']=monthly[['WaterMonth','PotentialPower4u']].groupby('WaterMonth').quantile(0.5)


calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,PotentialPower1u_P95,PotentialPower1u_P50,PotentialPower2u_P95,PotentialPower2u_P50,PotentialPower3u_P95,PotentialPower3u_P50,PotentialPower4u_P95,PotentialPower4u_P50,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.578553,0.396933,0.400655,0.274923,120.0,93.623309,97.000000,52.0,...,56.910351,57.310770,113.307225,114.141934,168.577230,169.914065,221.033039,223.017374,7042.548593,0.300778
2,Nov,11,0.403429,0.331518,0.334202,0.218705,114.0,89.358423,93.000000,50.0,...,56.541697,57.216376,112.541689,113.945052,167.358330,169.598481,219.239927,222.548335,8967.171547,0.306616
3,Dec,12,0.288584,0.241700,0.242757,0.157837,126.0,100.289629,104.000000,59.0,...,54.656780,56.573914,108.654611,112.609375,161.234986,167.468007,210.381481,219.405604,14425.402351,0.275372
4,Jan,1,0.222432,0.177347,0.181308,0.097665,150.0,117.763441,120.000000,73.0,...,49.863356,55.144085,98.888691,109.655672,146.142000,162.802530,189.218327,212.627486,20486.521889,0.244169
5,Feb,2,0.188832,0.129466,0.139340,0.023961,197.0,138.959744,137.000000,85.0,...,36.974507,52.364643,72.867734,103.966843,106.518836,153.945880,135.056354,200.056621,18662.406260,0.190080
6,Mar,3,0.182992,0.095462,0.091525,0.040246,315.0,194.510926,185.451613,107.0,...,24.629956,42.300740,48.147211,83.573192,60.544330,122.701095,28.147306,156.889009,30306.066543,0.292099
7,Apr,4,0.165295,0.085109,0.082042,0.049993,364.0,231.265233,217.000000,125.0,...,26.860099,38.282612,52.599160,75.487465,76.035544,110.454761,85.432139,140.307947,26305.577811,0.267540
8,May,5,0.223150,0.130801,0.128302,0.066393,340.0,265.271245,276.000000,147.0,...,34.433058,43.059562,67.761532,85.106476,98.805398,125.039232,124.662022,160.093269,16447.349603,0.144157
9,Jun,6,0.369425,0.200040,0.195288,0.108731,293.0,212.942652,210.000000,119.0,...,44.935777,50.882517,88.896653,100.944728,130.816602,149.269075,168.004965,193.484205,23185.985098,0.220721


In [55]:
calmonthly['PropYearsMinAvailFlowGt1UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGt2UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit*2).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGt3UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit*3).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGtCanalRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>canal_capacity).groupby('WaterMonth').count()/len(yearly.index)
calmonthly.fillna(0)
calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,PotentialPower3u_P95,PotentialPower3u_P50,PotentialPower4u_P95,PotentialPower4u_P50,Energy_stdev,Energy_coefvar,PropYearsMinAvailFlowGt1UnitRatedFlow,PropYearsMinAvailFlowGt2UnitRatedFlow,PropYearsMinAvailFlowGt3UnitRatedFlow,PropYearsMinAvailFlowGtCanalRatedFlow
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.578553,0.396933,0.400655,0.274923,120.0,93.623309,97.000000,52.0,...,168.577230,169.914065,221.033039,223.017374,7042.548593,0.300778,0.032258,NaN,NaN,NaN
2,Nov,11,0.403429,0.331518,0.334202,0.218705,114.0,89.358423,93.000000,50.0,...,167.358330,169.598481,219.239927,222.548335,8967.171547,0.306616,0.053763,NaN,NaN,NaN
3,Dec,12,0.288584,0.241700,0.242757,0.157837,126.0,100.289629,104.000000,59.0,...,161.234986,167.468007,210.381481,219.405604,14425.402351,0.275372,0.279570,0.021505,NaN,NaN
4,Jan,1,0.222432,0.177347,0.181308,0.097665,150.0,117.763441,120.000000,73.0,...,146.142000,162.802530,189.218327,212.627486,20486.521889,0.244169,0.967742,0.215054,0.043011,NaN
5,Feb,2,0.188832,0.129466,0.139340,0.023961,197.0,138.959744,137.000000,85.0,...,106.518836,153.945880,135.056354,200.056621,18662.406260,0.190080,1.000000,0.838710,0.344086,0.129032
6,Mar,3,0.182992,0.095462,0.091525,0.040246,315.0,194.510926,185.451613,107.0,...,60.544330,122.701095,28.147306,156.889009,30306.066543,0.292099,1.000000,0.978495,0.795699,0.559140
7,Apr,4,0.165295,0.085109,0.082042,0.049993,364.0,231.265233,217.000000,125.0,...,76.035544,110.454761,85.432139,140.307947,26305.577811,0.267540,1.000000,1.000000,0.967742,0.806452
8,May,5,0.223150,0.130801,0.128302,0.066393,340.0,265.271245,276.000000,147.0,...,98.805398,125.039232,124.662022,160.093269,16447.349603,0.144157,1.000000,0.946237,0.892473,0.720430
9,Jun,6,0.369425,0.200040,0.195288,0.108731,293.0,212.942652,210.000000,119.0,...,130.816602,149.269075,168.004965,193.484205,23185.985098,0.220721,0.946237,0.559140,0.258065,0.053763


In [56]:
calmonthly['PropYears1uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower1u']].where(lambda x: x['PotentialPower1u']>rated_power_unit).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears2uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower2u']].where(lambda x: x['PotentialPower2u']>rated_power_unit*2).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears3uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower3u']].where(lambda x: x['PotentialPower3u']>rated_power_unit*3).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears4uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower4u']].where(lambda x: x['PotentialPower4u']>yearly['Power_max'].max()).groupby('WaterMonth').count()/len(yearly.index)

calmonthly.fillna(0)
calmonthly


,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_stdev,Energy_coefvar,PropYearsMinAvailFlowGt1UnitRatedFlow,PropYearsMinAvailFlowGt2UnitRatedFlow,PropYearsMinAvailFlowGt3UnitRatedFlow,PropYearsMinAvailFlowGtCanalRatedFlow,PropYears1uPotentialGtRatedPower,PropYears2uPotentialGtRatedPower,PropYears3uPotentialGtRatedPower,PropYears4uPotentialGtRatedPower
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.578553,0.396933,0.400655,0.274923,120.0,93.623309,97.000000,52.0,...,7042.548593,0.300778,0.032258,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000
2,Nov,11,0.403429,0.331518,0.334202,0.218705,114.0,89.358423,93.000000,50.0,...,8967.171547,0.306616,0.053763,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000
3,Dec,12,0.288584,0.241700,0.242757,0.157837,126.0,100.289629,104.000000,59.0,...,14425.402351,0.275372,0.279570,0.021505,NaN,NaN,1.000000,1.000000,1.000000,1.000000
4,Jan,1,0.222432,0.177347,0.181308,0.097665,150.0,117.763441,120.000000,73.0,...,20486.521889,0.244169,0.967742,0.215054,0.043011,NaN,0.956989,0.946237,0.935484,0.903226
5,Feb,2,0.188832,0.129466,0.139340,0.023961,197.0,138.959744,137.000000,85.0,...,18662.406260,0.190080,1.000000,0.838710,0.344086,0.129032,0.741935,0.731183,0.698925,0.645161
6,Mar,3,0.182992,0.095462,0.091525,0.040246,315.0,194.510926,185.451613,107.0,...,30306.066543,0.292099,1.000000,0.978495,0.795699,0.559140,0.311828,0.301075,0.290323,0.236559
7,Apr,4,0.165295,0.085109,0.082042,0.049993,364.0,231.265233,217.000000,125.0,...,26305.577811,0.267540,1.000000,1.000000,0.967742,0.806452,0.150538,0.129032,0.096774,0.075269
8,May,5,0.223150,0.130801,0.128302,0.066393,340.0,265.271245,276.000000,147.0,...,16447.349603,0.144157,1.000000,0.946237,0.892473,0.720430,0.161290,0.161290,0.129032,0.118280
9,Jun,6,0.369425,0.200040,0.195288,0.108731,293.0,212.942652,210.000000,119.0,...,23185.985098,0.220721,0.946237,0.559140,0.258065,0.053763,0.677419,0.645161,0.559140,0.483871


## Flow duration curve

In [57]:
daily['ExceedanceRnd']=np.round(daily['Exceedance']*2,2)/2
fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

,Flow,Exceedance,Volume,Flow_difference,Flow_difference_pct,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,PotentialPower1u,PotentialPower2u,PotentialPower3u,PotentialPower4u
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,8865.175953,0.001412,0.765951,137.615867,0.015867,0.026324,1.102941,65.735294,12.338235,14.691176,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.005,6478.790089,0.004989,0.559767,115.269934,0.017456,0.041149,1.091954,72.436782,13.850575,17.701149,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.924772,23.278720,20.571241,0.000000
0.010,5544.712707,0.010047,0.479063,68.659047,0.012349,0.077041,1.289941,72.680473,13.656805,17.674556,...,9.617783,9.617783,9.617783,38.471131,923.307133,0.007106,24.939868,48.763692,70.299113,43.046903
0.015,5190.964012,0.014953,0.448499,88.798283,0.017178,0.105349,1.401163,71.540698,12.941860,16.645349,...,22.887085,22.887085,22.887085,91.548338,2197.160113,0.017651,26.403864,51.686017,74.665051,92.482740
0.020,4777.593171,0.020076,0.412784,113.350030,0.023656,0.110465,1.406977,69.976744,13.156977,16.616279,...,24.716775,24.716775,24.716775,98.867101,2372.810412,0.020720,28.255700,55.385964,80.201600,99.814393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,185.643897,0.979851,0.016040,2.397496,0.012948,0.897619,4.511905,28.017857,4.333333,2.708333,...,NaN,NaN,NaN,26.263488,630.323719,0.141422,57.660533,114.874101,171.094043,224.785539
0.985,175.260665,0.985178,0.015143,1.918049,0.010903,0.920385,4.682692,26.668269,4.254808,2.524038,...,NaN,NaN,NaN,24.754930,594.118324,0.141300,57.636643,114.823985,171.013024,224.663568
0.990,165.025295,0.990459,0.014258,2.091723,0.012641,0.944222,4.859259,25.370370,4.251852,2.325926,...,NaN,NaN,NaN,23.264526,558.348614,0.140952,57.615595,114.779846,170.941698,224.556260


## Add overall summary data to the model file

In [58]:

models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

models.at[model_name,'Power_max']=yearly['Power_max'].max()
models.at[model_name,'Power_Turbine1_max']=yearly['PowerTurbine1_max'].max()

models.at[model_name,'CapFactor_Power_max']=calyearly['Energy'].mean()/(yearly['Power_max'].max()*24*365)

models.at[model_name,'RatedPower']=rated_power_unit

models.loc[model_name]



OutputPrefix                            base
Description                        Base Case
EWRCatSet                   Recommendation 1
HeadpondLift                               0
Capacity                                 180
MinimumHead                               10
MinimumFlowUnit                           50
FixedTailwater                             0
UnitsAvailable                             4
RatedDischarge                           250
RatedHead                               21.7
RatedTurbineEfficiency                 0.943
RatedGeneratorEfficiency                0.98
MaximumLoadFactor                        1.1
RatedPower                            49.182
Mean                                  832484
P95                                   718504
P90                                   756610
P75                                   792216
P50                                   838165
P25                                   876248
P10                                   921915
P5        

In [59]:
autocorrel=pd.DataFrame(statools.acf(yearly['Energy'],fft=True,qstat=False,alpha=0.05))
autocorrel

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1,0.488341,0.361535,0.288847,0.280791,0.18537,0.150983,0.10388,0.180448,0.115152,...,-0.178253,-0.0835501,-0.0180572,0.0627404,0.028512,-0.023787,-0.0064635,0.0295426,0.011441,0.0520337
1,"[1.0, 1.0]","[0.28510260314025293, 0.6915801982161341]","[0.11453860605024077, 0.6085309205338875]","[0.020881790872404493, 0.556811273153114]","[0.0002604174251233937, 0.5613223202799864]","[-0.10653971072891458, 0.47727895808166443]","[-0.1457491978097211, 0.44771441585473626]","[-0.19600811856551437, 0.40376839961704303]","[-0.1209233691078754, 0.48181850310362906]","[-0.19064942169950955, 0.4209530565106587]",...,"[-0.565210393993805, 0.20870389207861975]","[-0.47388427762604535, 0.3067840593575817]","[-0.4091293509524321, 0.3730149995498764]","[-0.328366257624769, 0.4538469687063973]","[-0.3630101338199342, 0.42003411233830057]","[-0.41539488330935126, 0.36782087480493186]","[-0.3981310519523266, 0.3852040605027886]","[-0.3621293742949508, 0.4212145498280478]","[-0.3803230318754776, 0.4032049553737764]","[-0.33974408578934423, 0.4438115031788127]"


In [60]:
lowheadshutdowns=daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowheadshutdowns['StartDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['Date']
lowheadshutdowns['EndDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').max()['Date']
lowheadshutdowns['MonthId']=daily[['WaterYear','MonthId']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['MonthId']

#daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').min()#['Date']
lowheadshutdowns['Month_power_max']=monthly.loc[lowheadshutdowns['MonthId']][['WaterYear','Power_max']].set_index('WaterYear')
lowheadshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
WaterYear,,,,,
1933,3,1934-03-26,1934-03-28,1934.03,180.964667
1947,31,1948-03-04,1948-04-03,1948.03,106.174038
1951,19,1952-03-02,1952-03-20,1952.03,104.964243
1952,9,1953-03-21,1953-03-29,1953.03,187.487855
1955,12,1956-05-05,1956-05-16,1956.05,115.164735
1956,27,1957-03-02,1957-03-28,1957.03,105.902482
1957,60,1958-02-03,1958-04-03,1958.02,110.164461
1960,22,1961-04-02,1961-04-23,1961.04,100.599028
1961,6,1962-04-05,1962-04-12,1962.04,104.188853


In [61]:
lowflowshutdowns=daily.loc[daily['LowFlowShutoff']==1].groupby('Year').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowflowshutdowns['StartDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['Date']
lowflowshutdowns['EndDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').max()['Date']
lowflowshutdowns['MonthId']=daily[['Year','MonthId']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['MonthId']


lowflowshutdowns['Month_power_max']=monthly.loc[lowflowshutdowns['MonthId']][['Year','Power_max']].set_index('Year')
lowflowshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
Year,,,,,
1924,10,1924-10-01,1924-10-10,1924.1,15.370217


## Save Files

In [62]:
outp=output_data+out_prefix
#daily.to_csv(outp+'_pe_daily.csv')
yearly.to_csv(outp+'_pe_yearly.csv')

In [32]:
outp=output_data+out_prefix

if dryrun==False:
    daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_pe_monthly.csv')
    yearly.to_csv(outp+'_pe_yearly.csv')
    edc.to_csv(outp+'_pe_percentiles.csv')
    monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    weekly.to_csv(outp+'_pe_weekly.csv')
    weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_pe_calmonthly.csv')
    power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    fdc.to_csv(outp+'_pe_fdc.csv')
    autocorrel.to_csv(outp+'_pe_autocorrel.csv')
    #models.to_csv(input_data + "models.csv")
    models.to_csv(input_data + "models.csv")
    lowheadshutdowns.to_csv(outp + "lowheadshutdowns.csv")
    lowflowshutdowns.to_csv(outp + "lowflowshutdowns.csv")

In [99]:
years=daily['WaterYear'].unique().tolist()

for year in years:
    days=daily.loc[daily.WaterYear==year]
    #days.to_csv(output_data + 'years/' + out_prefix + '_pe_daily_' + str(year) + '.csv')
 